In [ ]:
# CSV 파일 경로 설정 및 데이터 로드
file_path = 'ver3.csv'  # 실제 경로로 교체하세요
df = pd.read_csv(file_path)
print(df.dtypes)

file_path = 'label.csv'  # 실제 경로로 교체하세요
labels_df = pd.read_csv(file_path)
print(labels_df.head(5))

# 라벨 텐서 생성
y = torch.tensor(labels_df['label'].values, dtype=torch.long)

# 피어슨 상관계수 행렬 계산
pearson_corr = df.corr(method='pearson')
print(pearson_corr)

# 상위 95%에 해당하는 값 계산 후 엣지 리스트 생성
corr_array = pearson_corr.values
np.fill_diagonal(corr_array, np.nan)
flattened_corr = corr_array.flatten()
valid_corr = flattened_corr[~np.isnan(flattened_corr)]
top_95_percent_value = np.percentile(valid_corr, 95)
edge_indices = np.argwhere(corr_array >= top_95_percent_value)
edge_list = [[int(edge[0]), int(edge[1])] for edge in edge_indices]
edge_tensor = torch.tensor(edge_list, dtype=torch.long).T

# PyTorch 텐서로 변환
tensor_data = torch.tensor(df.values, dtype=torch.float32)  # 결측값 처리가 없이 원본 데이터 사용

# 노드 수 계산 및 마스크 생성
num_nodes = y.size(0)
train_mask = torch.ones(num_nodes, dtype=torch.bool)
test_mask = torch.ones(num_nodes, dtype=torch.bool)

# 데이터 객체 생성
data = Data(x=tensor_data, edge_index=edge_tensor, y=y, train_mask=train_mask, test_mask=test_mask)
print(data)

# RandomNodeSplit을 사용하여 데이터셋을 80% 트레인, 20% 밸리데이션으로 분할
split = T.RandomNodeSplit(num_val=0.2)
graph = split(data)

# GPU 사용 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
graph = graph.to(device)
num_classes = y.unique().size(0)

# GAT 모델 정의
class GAT(nn.Module):
    def __init__(self):
        super().__init__()
        self.convs = ModuleList([
            GATConv(data.num_node_features, 8, heads=8, dropout=0.8),
            GATConv(8 * 8, num_classes, heads=1, concat=False, dropout=0.8),
        ])

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i != len(self.convs) - 1:
                x = F.relu(x)
                x = F.dropout(x, p=0.8, training=self.training)
        return F.log_softmax(x, dim=-1)

# 학습 함수
def train_node_classifier(model, graph, optimizer, criterion, n_epochs=500):
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph)
        loss = criterion(out[graph.train_mask], graph.y[graph.train_mask])
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch: {epoch:03d}, Train Loss: {loss:.3f}')

    return model

# 평가 함수
def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()

    acc = int(correct) / int(mask.sum())
    return acc

# 모델 학습 및 평가
gcn = GAT().to(device)
optimizer_gcn = torch.optim.Adam(gcn.parameters(), lr=0.001, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
gcn = train_node_classifier(gcn, graph, optimizer_gcn, criterion)

val_acc = eval_node_classifier(gcn, graph, graph.val_mask)
#print(f'Validation Accuracy: {val_acc:.3f}')

##############################
# Val 데이터 적용 및 예측 부분 #
##############################

# 1. Val 데이터셋 불러오기
val_df = pd.read_csv('val_ver4.csv')  # 실제 경로로 교체하세요
val_labels_df = pd.read_csv('val_label.csv')  # 실제 경로로 교체하세요

# 2. 라벨 텐서 생성
val_y = torch.tensor(val_labels_df['label'].values, dtype=torch.long)

# 3. PyTorch 텐서로 변환
val_tensor_data = torch.tensor(val_df.values, dtype=torch.float32)

# 4. Val 데이터셋에 대해 피어슨 상관계수 행렬 계산 및 엣지 리스트 생성
val_pearson_corr = val_df.corr(method='pearson').abs()
val_corr_array = val_pearson_corr.values
np.fill_diagonal(val_corr_array, np.nan)
val_edge_indices = np.argwhere(val_corr_array >= top_95_percent_value)
val_edge_list = [[int(edge[0]), int(edge[1])] for edge in val_edge_indices]
val_edge_tensor = torch.tensor(val_edge_list, dtype=torch.long).T

# 5. PyTorch Geometric의 Data 객체 생성
val_data = Data(x=val_tensor_data, edge_index=val_edge_tensor, y=val_y)

# 6. GPU로 데이터 이동 (모델과 같은 디바이스로 이동)
val_data = val_data.to(device)

# 7. Val 데이터셋에 대해 모델 예측 수행
gcn.eval()  # 평가 모드로 전환
with torch.no_grad():
    val_pred = gcn(val_data).argmax(dim=1)  # 예측 값

# 8. 실제 레이블과 예측 값 비교
correct_val = (val_pred == val_data.y).sum()
val_accuracy = int(correct_val) / val_data.y.size(0)

# 결과 출력
print(f'Val 데이터셋에 대한 예측 정확도: {val_accuracy:.3f}')
print(f'Val 데이터셋 실제 레이블: {val_data.y}')
print(f'Val 데이터셋 예측된 레이블: {val_pred}')
